# Projeto final

### Ironhack - Data Analytics Bootcamp

### Fernanda Harumi Tanaka

***

## Índice:

[Introdução](#Introdução) <br>
[Metodologia](#Metodologia) <br>
> [1. Análise do dataset](#1.-Análise-do-dataset:) <br>
> [2. Mapa astral](#2.-Mapa-astral:) <br>
> [3. Cálculo de distâncias](#3.-Cálculo-de-distâncias:) <br>
>>[3.1 Por similaridade](#3.1-Por-similaridade:) <br>
>>[3.2 Por compatibilidade](#3.2-Por-compatibilidade:) <br>

> [4. Interação com os dados do usuário](#4.-Interação-com-os-dados-do-usuário:)


***

## Introdução:

O principal objetivo deste projeto é identificar quais famosos mais se assemelham e são compatíveis com a personalidade do usuário, de acordo com os seus mapas astrais. <br>
Para tanto, foi realizado webscraping do site _https://www.astro.com/astro-databank/Main_Page_, que reúne dados astrológicos de acontecimentos marcantes e personalidades da mídia.

***

## Metodologia:

In [1]:
import pandas as pd
import flatlib
import geopy
import pyaztro
import pytz
import re

In [2]:
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib.chart import Chart
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from pytz import timezone
from datetime import datetime
from sklearn.neighbors import BallTree

In [3]:
class pessoa():
    """
    Essa classe calcula o mapa astral de uma pessoa. Forneça como input seu nome, data de nascimento, horário, fuso, local de nascimento (ex: "São Paulo, SP").
    """
       
    def __init__(self, nome, data, horario, fuso, local):
        self.nome = nome
        self.data = data
        self.horario = horario
        self.fuso = fuso
        self.local = local
        self.chart = None
        
    def mapa(self):
        geolocator = Nominatim(user_agent="my-application")
        location = geolocator.geocode(self.local)
        mapa_pessoa = Chart(Datetime(self.data, self.horario, self.fuso), GeoPos(location.latitude, location.longitude)).objects
        dicionario = {"nome":self.nome}
        for item in mapa_pessoa:
            dicionario[item.id] = item.sign
        self.chart = dicionario

In [4]:
def drop_word (df, word):
    """
    Essa função elimina as linhas correspondentes à determinada palavra presente na coluna "bio".
    Insira o nome do dataframe (df) e a palavra que deseja buscar (word). 
    Retorna o dataset sem linhas que contenham essa palavra.
    """
    df.drop(df.loc[df['bio'].str.contains(word)].index, axis=0, inplace=True)
    return df

In [5]:
tf = TimezoneFinder()
utc = pytz.utc

def offset(df):
    """
    Essa função retorna o fuso horário de um local, de acordo com o deslocamento UTC, em horas.
    Insira um dataframe (df), que contenha as colunas lat_2 e long_2 (em float).
    Função retirada da documentação do módulo TimezoneFinder "https://github.com/MrMinimal64/timezonefinder"
    """
    
    today = datetime.now()
    try:
        tz_target = timezone(tf.certain_timezone_at(lat=df['lat_2'], lng=df['long_2']))
    
        today_target = tz_target.localize(today)
        today_utc = utc.localize(today)
        offset = today_utc - today_target
    
        offset -= tz_target.dst(today)
        return offset.total_seconds() / 3600

    except:
        pass

In [6]:
def mapa(df):
    """
    Essa função retorna o mapa astral em um dicionário 
    """
    mapa_pessoa = Chart(Datetime(df['data'], df['horario'], df['fuso']), GeoPos(df['lat_2'], df['long_2'])).objects
    dicionario = {}
    for item in mapa_pessoa:
        dicionario[item.id] = item.sign
    return dicionario

### 1. Análise do dataset:

In [7]:
# Importando o dataset:
astro = pd.read_csv("astro_databank.csv")
astro.columns = ['nome', 'data', 'horario', 'lat', 'long', 'bio', 'url']

In [8]:
astro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65417 entries, 0 to 65416
Data columns (total 7 columns):
nome       65417 non-null object
data       65417 non-null object
horario    65417 non-null object
lat        65417 non-null object
long       65417 non-null object
bio        65417 non-null object
url        65417 non-null object
dtypes: object(7)
memory usage: 3.5+ MB


In [9]:
astro.head()

,nome,data,horario,lat,long,bio,url
0,"Bablet, Denis",19 March 1930,01:30,48n52,2e20,<p>French theorist of the theatre. \n</p>,"https://www.astro.com/astro-databank/Bablet,_D..."
1,"Babin Cortes, Maria Teresa",31 May 1910,05:30,18n01,66w37,"<p>Puerto Rican educator, literary critic, and...",https://www.astro.com/astro-databank/Babin_Cor...
2,"Babonneau, René",18 June 1904,09:30,47n13,1w33,"<p>French legionnaire, awarded the title of 'C...",https://www.astro.com/astro-databank/Babonneau...
3,"Babonneau, René",18 June 1904,09:30,47n13,1w33,"<p>French legionnaire, awarded the title of 'C...",https://www.astro.com/astro-databank/Babonneau...
4,"Baboul, Eudoxie",1 October 1901,03:00,5n23,52w57,"<p>French supercentenarian, for a while the ol...","https://www.astro.com/astro-databank/Baboul,_E..."


"Babonneau, René" é um dado duplicado. Deve-se eliminar esses dados:

In [10]:
astro2 = astro.drop_duplicates()

In [11]:
astro2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57706 entries, 0 to 58480
Data columns (total 7 columns):
nome       57706 non-null object
data       57706 non-null object
horario    57706 non-null object
lat        57706 non-null object
long       57706 non-null object
bio        57706 non-null object
url        57706 non-null object
dtypes: object(7)
memory usage: 3.5+ MB


In [12]:
astro2.head()

,nome,data,horario,lat,long,bio,url
0,"Bablet, Denis",19 March 1930,01:30,48n52,2e20,<p>French theorist of the theatre. \n</p>,"https://www.astro.com/astro-databank/Bablet,_D..."
1,"Babin Cortes, Maria Teresa",31 May 1910,05:30,18n01,66w37,"<p>Puerto Rican educator, literary critic, and...",https://www.astro.com/astro-databank/Babin_Cor...
2,"Babonneau, René",18 June 1904,09:30,47n13,1w33,"<p>French legionnaire, awarded the title of 'C...",https://www.astro.com/astro-databank/Babonneau...
3,"Babonneau, René",18 June 1904,09:30,47n13,1w33,"<p>French legionnaire, awarded the title of 'C...",https://www.astro.com/astro-databank/Babonneau...
4,"Baboul, Eudoxie",1 October 1901,03:00,5n23,52w57,"<p>French supercentenarian, for a while the ol...","https://www.astro.com/astro-databank/Baboul,_E..."


In [13]:
len(astro)-len(astro2)

7711

7711 dados duplicados foram eliminados, porém o "Babonneau, René" continuou com duas ocorrências no dataset. Assim como este caso, devem existir outros. Então, eliminou-se pessoas homônimas:

In [14]:
astro3 = astro2.drop_duplicates(subset = "nome")

In [15]:
astro3.head()

,nome,data,horario,lat,long,bio,url
0,"Bablet, Denis",19 March 1930,01:30,48n52,2e20,<p>French theorist of the theatre. \n</p>,"https://www.astro.com/astro-databank/Bablet,_D..."
1,"Babin Cortes, Maria Teresa",31 May 1910,05:30,18n01,66w37,"<p>Puerto Rican educator, literary critic, and...",https://www.astro.com/astro-databank/Babin_Cor...
2,"Babonneau, René",18 June 1904,09:30,47n13,1w33,"<p>French legionnaire, awarded the title of 'C...",https://www.astro.com/astro-databank/Babonneau...
4,"Baboul, Eudoxie",1 October 1901,03:00,5n23,52w57,"<p>French supercentenarian, for a while the ol...","https://www.astro.com/astro-databank/Baboul,_E..."
5,"Babilée, Jean",3 February 1923,21:30,48n5102,2e1958,"<p>French dancer, choreographer, and actor, Ba...",https://www.astro.com/astro-databank/Babil%C3%...


In [16]:
astro3.reset_index(inplace=True)
astro3.drop("index", axis=1, inplace=True)

C:\Users\ferna\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [17]:
astro3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52548 entries, 0 to 52547
Data columns (total 7 columns):
nome       52548 non-null object
data       52548 non-null object
horario    52548 non-null object
lat        52548 non-null object
long       52548 non-null object
bio        52548 non-null object
url        52548 non-null object
dtypes: object(7)
memory usage: 2.8+ MB


In [18]:
len(astro2)-len(astro3)

5158

Com isso, mais de 5000 dados homônimos foram eliminados. O dataset contém, desta forma, 52548 elementos com uma única ocorrência, sem homônimos, e sem valores nulos.

Antes de filtrar o dataset, foram separados alguns dados interessantes, que podem ser utilizados em etapas futuras do projeto.

In [19]:
#Separando o dataset para dados de acidentes
mask_accident = astro3['nome'].str.contains('Accident')
accident = astro3[mask_accident]
accident.head()

,nome,data,horario,lat,long,bio,url
303,Accidents: West Fertilizer Company explosion (...,17 April 2013,07:50:38,31n33,97w09,"<p>On April 17, 2013, an ammonium nitrate expl...",https://www.astro.com/astro-databank/Accidents...
304,Accident: wreck Air France flight 447,1 June 2009,05:14,3s5025,32w2439,<p>This plane (Rio-Paris) sank into the Atlant...,https://www.astro.com/astro-databank/Accident:...
306,Accident: great fire of Valparaiso,12 April 2014,16:40,33s02,71w38,"<p>Great wildfire, it began on 12 April 2914 i...",https://www.astro.com/astro-databank/Accident:...
307,Accident: gas explosion Mulhouse,31 August 2014,07:00,47n45,7e20,<p><br>\n</p>,https://www.astro.com/astro-databank/Accident:...
311,Accident: Train Wreck,25 April 1946,13:15,41n47,88w09,"<p>American train accident; the train, called ...",https://www.astro.com/astro-databank/Accident:...


In [20]:
accident.shape

(94, 7)

In [21]:
#Separando dados relativos ao Brasil:
br_mask = astro3['bio'].str.contains('Brazilian')
br = astro3[br_mask]
br.head()

,nome,data,horario,lat,long,bio,url
123,UFO Sighting 14368,10 February 1969,21:00,21s1218,41w5316,<p>Brazilian UFO sighting of a pear-shaped cra...,https://www.astro.com/astro-databank/UFO_Sight...
124,UFO Sighting 14367,8 February 1969,21:05,16s0456,48w3026,"<p>Brazilian UFO sighting of a ""wheel of fire""...",https://www.astro.com/astro-databank/UFO_Sight...
127,UFO Sighting 14366,6 February 1969,07:30,21s59,47w25,"<p>Brazilian UFO sighting, an encounter of the...",https://www.astro.com/astro-databank/UFO_Sight...
128,UFO Sighting 14365,31 January 1969,22:00,16s0456,48w3026,"<p>Brazilian UFO sighting, an encounter of the...",https://www.astro.com/astro-databank/UFO_Sight...
132,"Salomão, Waly",3 September 1943,05:00,13s5127,40w0501,"<p>Brazilian poet, songwriter and writer. His ...",https://www.astro.com/astro-databank/Salom%C3%...


In [22]:
br.shape

(627, 7)

In [23]:
#Separando dados relativos a UFO Sightings:
ufo_mask = astro3['nome'].str.contains('UFO Sighting')
ufo = astro3[ufo_mask]
ufo.head()

,nome,data,horario,lat,long,bio,url
123,UFO Sighting 14368,10 February 1969,21:00,21s1218,41w5316,<p>Brazilian UFO sighting of a pear-shaped cra...,https://www.astro.com/astro-databank/UFO_Sight...
124,UFO Sighting 14367,8 February 1969,21:05,16s0456,48w3026,"<p>Brazilian UFO sighting of a ""wheel of fire""...",https://www.astro.com/astro-databank/UFO_Sight...
127,UFO Sighting 14366,6 February 1969,07:30,21s59,47w25,"<p>Brazilian UFO sighting, an encounter of the...",https://www.astro.com/astro-databank/UFO_Sight...
128,UFO Sighting 14365,31 January 1969,22:00,16s0456,48w3026,"<p>Brazilian UFO sighting, an encounter of the...",https://www.astro.com/astro-databank/UFO_Sight...
554,UFO Sighting 14363,26 January 1969,21:22,12s0808,38w2509,"<p>Brazilian UFO sighting, an encounter of the...",https://www.astro.com/astro-databank/UFO_Sight...


In [24]:
ufo.shape

(482, 7)

In [25]:
entert = astro3[astro3['nome'].str.contains('Entertainment')]
entert.head()

,nome,data,horario,lat,long,bio,url
715,Entertainment: Tarzan,1 December 1911,20:00,41n51,87w39,<p>American fictional character created by Edg...,https://www.astro.com/astro-databank/Tarzan
16518,Entertainment: French and Saunders,9 March 1987,21:00,51n30,0w10,<p>British sketch comedy television series wri...,https://www.astro.com/astro-databank/Entertain...
16519,Entertainment: Eurovision Song Contest,24 May 1956,21:00,46n01,8e58,"<p>International TV song competition, the long...",https://www.astro.com/astro-databank/Entertain...
16521,Entertainment: Eight Is Enough,15 March 1977,20:30,40n43,74w00,<p>American television comedy-drama series tha...,https://www.astro.com/astro-databank/Entertain...
16524,Entertainment: Dilwale Dulhania Le Jayenge,20 October 1995,12:00,18n58,72e50,<p>Indian romance film written and directed by...,https://www.astro.com/astro-databank/Entertain...


In [26]:
entert.shape

(144, 7)

In [27]:
nature = astro3[astro3['nome'].str.contains('Nature')]
nature.head()

,nome,data,horario,lat,long,bio,url
105,Nature:Earthquake California 1812 (a) Wrightwood,8 December 1812,07:00,34n22,117w38,<p>Historical earthquake in California. Estima...,https://www.astro.com/astro-databank/Nature:Ea...
107,Nature:Earthquake California/Mexico: 1979,15 October 1979,23:16:57,32n59,115w32,<p>Californian-mexican earthquake of magnitude...,https://www.astro.com/astro-databank/Nature:Ea...
108,Nature:Earthquake Burma (Myanmar) 1908,12 December 1908,12:55,26n27,96e42,<p>Historical Burma (Myanmar) earthquake. Est...,https://www.astro.com/astro-databank/Nature:Ea...
110,Nature:Earthquake Bulgaria 1928,18 April 1928,19:22,42n09,24e45,<p>Historical Bulgaria earthquake. Estimated ...,https://www.astro.com/astro-databank/Nature:Ea...
111,Nature:Earthquake Bangladesh 1762 (Arakan),22 March 1762,11:00,22n20,91e50,<p>Historical earthquake in Bangladesh (then i...,https://www.astro.com/astro-databank/Nature:Ea...


In [28]:
nature.shape

(623, 7)

In [29]:
plane = astro3[astro3['nome'].str.contains('Plane Crash')]
plane.head()

,nome,data,horario,lat,long,bio,url
305,Plane Crash: AirAsia Flight 8501 (2014),28 December 2014,05:35,2s08,106e08,<p><br>\n</p>,https://www.astro.com/astro-databank/Accident:...
904,Plane Crash: TWA Flight 800 (1964),23 November 1964,14:15,41n54,12e29,"<p>On November 23, 1964 at 02:09 p.m. the capt...",https://www.astro.com/astro-databank/Accident:...
1611,Plane Crash: Frontier Airlines (1964),12 March 1964,20:50,46n25,105w50,"<p>On March 12, 1964, at 8:50 p.m. Flight 32 o...",https://www.astro.com/astro-databank/Accident:...
1612,Plane Crash: EAL Flight 304 (1964),25 February 1964,02:06,30n17,89w47,<p>Eastern Air Lines Flight 304 was a Douglas ...,https://www.astro.com/astro-databank/Accident:...
1615,Accident: Plane Crash 14268,21 December 1967,16:00,39n44,105w01,"<p>Zip Dobyns, news source unrecorded\n</p>",https://www.astro.com/astro-databank/Accident:...


In [30]:
plane.shape

(176, 7)

In [31]:
animal = astro3[astro3['nome'].str.contains('Animal')]
animal.head()

,nome,data,horario,lat,long,bio,url
14242,Animal: Ruffian,17 April 1972,21:50,38n13,84w15,"<p>American champion thoroughbred racehorse, w...",https://www.astro.com/astro-databank/Ruffian
46714,Animal: Orkid (killer whale),23 September 1988,16:30,32n43,117w09,"<p>American killer whale, later nicknamed 'The...",https://www.astro.com/astro-databank/Animal:_W...
46717,Animal: Secretariat,30 March 1970,00:10,37n52,77w28,<p>American race horse known as one of the bes...,https://www.astro.com/astro-databank/Animal:_S...
46719,Animal: Rockstardom,17 August 2007,03:16,32s30,150e55,"<p>Australia thoroughbred racehorse, the first...",https://www.astro.com/astro-databank/Animal:_R...
46720,Animal: Sea Pigeon,7 March 1970,04:00,38n03,84w30,"<p>American-bred, British-trained racehorse wh...",https://www.astro.com/astro-databank/Animal:_S...


In [32]:
animal.shape

(23, 7)

In [33]:
hist = astro3[astro3['nome'].str.contains('Historic')]
hist.head()

,nome,data,horario,lat,long,bio,url
3085,Historic: Internet,29 October 1969,22:30,34n03,118w15,<p><br>\n</p>,https://www.astro.com/astro-databank/Internet
13851,"Historic: Sarajevo, reunification",19 March 1996,06:00,43n52,18e25,<p>Reunification of Sarajevo after the NATO ha...,"https://www.astro.com/astro-databank/Sarajevo,..."
29727,Historic: World Trade Center,4 April 1973,15:00,40n46,73w59,<p>American trade buildings standing out in th...,https://www.astro.com/astro-databank/Historic:...
29729,Historic: World War II,1 September 1939,04:45,54n23,18e40,"<p>Global war that lasted from 1939 to 1945, a...",https://www.astro.com/astro-databank/Historic:...
29730,Historic: World's Fair,30 April 1939,15:12,40n43,74w00,<p>American New York World's Fair opened with ...,https://www.astro.com/astro-databank/Historic:...


In [34]:
hist.shape

(94, 7)

Feito essas separações, o dataset foi filtrado, com a intenção de manter personalidades famosas para o público brasileiro.

Primeiramente, eliminou-se os já conhecidos acontecimentos relativos a "Accidents", "UFO Sighting", "Entertainment", "Plane Crash", "Animal" e "Historic":

In [35]:
astro4 = astro3[astro3['nome'].str.contains('UFO Sighting') == False][astro3['nome'].str.contains('Accident') == False][astro3['nome'].str.contains('Entertainment') == False][astro3['nome'].str.contains('Nature') == False][astro3['nome'].str.contains('Plane Crash') == False][astro3['nome'].str.contains('Animal') == False][astro3['nome'].str.contains('Historic') == False]

C:\Users\ferna\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [36]:
astro4.head()

,nome,data,horario,lat,long,bio,url
0,"Bablet, Denis",19 March 1930,01:30,48n52,2e20,<p>French theorist of the theatre. \n</p>,"https://www.astro.com/astro-databank/Bablet,_D..."
1,"Babin Cortes, Maria Teresa",31 May 1910,05:30,18n01,66w37,"<p>Puerto Rican educator, literary critic, and...",https://www.astro.com/astro-databank/Babin_Cor...
2,"Babonneau, René",18 June 1904,09:30,47n13,1w33,"<p>French legionnaire, awarded the title of 'C...",https://www.astro.com/astro-databank/Babonneau...
3,"Baboul, Eudoxie",1 October 1901,03:00,5n23,52w57,"<p>French supercentenarian, for a while the ol...","https://www.astro.com/astro-databank/Baboul,_E..."
4,"Babilée, Jean",3 February 1923,21:30,48n5102,2e1958,"<p>French dancer, choreographer, and actor, Ba...",https://www.astro.com/astro-databank/Babil%C3%...


In [37]:
astro4.reset_index(inplace=True)
astro4.drop("index", axis=1, inplace=True)

In [38]:
astro4.shape

(50913, 7)

Excluiu-se alguns dados, principalmente referentes a profissões e nacionalidades com poucas ocorrências:

In [39]:
drop_word(astro4, "theorist")
drop_word(astro4, "sculptor")
drop_word(astro4, "lawyer")
drop_word(astro4, "cyclist")
drop_word(astro4, "critic")
drop_word(astro4, "banker")
drop_word(astro4, "hockey")
drop_word(astro4, "skater")
drop_word(astro4, "harpist")
drop_word(astro4, "jockey")
drop_word(astro4, "historian")
drop_word(astro4, "army")
drop_word(astro4, "diplomat")
drop_word(astro4, "supercentenarian")
drop_word(astro4, "legionnaire")
drop_word(astro4, "Turkish")
drop_word(astro4, "Finnish")
drop_word(astro4, "Sri Lankan")
drop_word(astro4, "Puerto")
drop_word(astro4, "Polish")
drop_word(astro4, "Belgian")
drop_word(astro4, "Nigerian")
drop_word(astro4, "French arc")

astro4.shape

(44500, 7)

In [40]:
astro4.reset_index(inplace=True)
astro4.drop("index", axis=1, inplace=True)

Após essa pequena limpeza, aproximadamente 6500 dados foram excluídos. Optou-se por seguir com o dataset dessa maneira.

Em seguida, realizou-se uma limpeza na coluna 'bio'.

In [41]:
astro4['bio'] = astro4['bio'].str.replace('<p>','')
astro4['bio'] = astro4['bio'].str.replace('</p>','')
astro4['bio'] = astro4['bio'].str.replace('\n','')

O próximo passo consistiu em transformar a coluna "data" no formato 'YYYY/MM/DD':

In [42]:
astro4['data'] = pd.to_datetime(astro4['data'], errors='coerce')

In [43]:
astro4.head()

,nome,data,horario,lat,long,bio,url
0,"Babilée, Jean",1923-02-03,21:30,48n5102,2e1958,"French dancer, choreographer, and actor, Babil...",https://www.astro.com/astro-databank/Babil%C3%...
1,"Baber, Alice",1928-08-22,03:40,39n30,88w11,"American artist, her style was abstract expres...","https://www.astro.com/astro-databank/Baber,_Alice"
2,"Babbitt, Manuel",1949-05-03,21:30,41n46,70w43,American homicide: a former Marine and Vietnam...,"https://www.astro.com/astro-databank/Babbitt,_..."
3,"Babbitt, Elwood",1921-11-26,10:20,42n35,72w19,American medium based in New England. His lif...,"https://www.astro.com/astro-databank/Babbitt,_..."
4,"A, Dominique",1968-10-06,17:35,48n33,3e18,"French songwriter, singer and guitarist.","https://www.astro.com/astro-databank/A,_Dominique"


In [44]:
astro4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44500 entries, 0 to 44499
Data columns (total 7 columns):
nome       44500 non-null object
data       43613 non-null datetime64[ns]
horario    44500 non-null object
lat        44500 non-null object
long       44500 non-null object
bio        44500 non-null object
url        44500 non-null object
dtypes: datetime64[ns](1), object(6)
memory usage: 2.4+ MB


In [45]:
astro4 = astro4.dropna()

In [46]:
# Convertendo o formato datetime64 para string:
astro4['data'] = astro4['data'].astype(str) 

In [47]:
astro4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43613 entries, 0 to 44499
Data columns (total 7 columns):
nome       43613 non-null object
data       43613 non-null object
horario    43613 non-null object
lat        43613 non-null object
long       43613 non-null object
bio        43613 non-null object
url        43613 non-null object
dtypes: object(7)
memory usage: 2.7+ MB


In [48]:
# Agora que já é uma string, pode-se aplicar métodos para adaptar ao formato desejado:
astro4['data'] = astro4['data'].str.replace('-','/')

In [49]:
astro4.head()

,nome,data,horario,lat,long,bio,url
0,"Babilée, Jean",1923/02/03,21:30,48n5102,2e1958,"French dancer, choreographer, and actor, Babil...",https://www.astro.com/astro-databank/Babil%C3%...
1,"Baber, Alice",1928/08/22,03:40,39n30,88w11,"American artist, her style was abstract expres...","https://www.astro.com/astro-databank/Baber,_Alice"
2,"Babbitt, Manuel",1949/05/03,21:30,41n46,70w43,American homicide: a former Marine and Vietnam...,"https://www.astro.com/astro-databank/Babbitt,_..."
3,"Babbitt, Elwood",1921/11/26,10:20,42n35,72w19,American medium based in New England. His lif...,"https://www.astro.com/astro-databank/Babbitt,_..."
4,"A, Dominique",1968/10/06,17:35,48n33,3e18,"French songwriter, singer and guitarist.","https://www.astro.com/astro-databank/A,_Dominique"


É necessário converter os dados de latitude e longitude para float. <br>
Exemplo: de '23s33' a -23.33 <br>
<br>
Latitude:

In [50]:
# Primeiro, os dados referentes ao sul, que devem ser negativos:
astro4.loc[(astro4['lat'].str.contains("s")),'lat_2'] = astro4['lat'].str.replace('s','.')
astro4['lat_2'] = "-" + astro4['lat_2']

In [51]:
# Depois, os dados referentes ao norte:
astro4.loc[(astro4['lat'].str.contains("n")),'lat_2'] = astro4['lat'].str.replace('n','.')

In [52]:
astro4.head()

,nome,data,horario,lat,long,bio,url,lat_2
0,"Babilée, Jean",1923/02/03,21:30,48n5102,2e1958,"French dancer, choreographer, and actor, Babil...",https://www.astro.com/astro-databank/Babil%C3%...,48.5102
1,"Baber, Alice",1928/08/22,03:40,39n30,88w11,"American artist, her style was abstract expres...","https://www.astro.com/astro-databank/Baber,_Alice",39.30
2,"Babbitt, Manuel",1949/05/03,21:30,41n46,70w43,American homicide: a former Marine and Vietnam...,"https://www.astro.com/astro-databank/Babbitt,_...",41.46
3,"Babbitt, Elwood",1921/11/26,10:20,42n35,72w19,American medium based in New England. His lif...,"https://www.astro.com/astro-databank/Babbitt,_...",42.35
4,"A, Dominique",1968/10/06,17:35,48n33,3e18,"French songwriter, singer and guitarist.","https://www.astro.com/astro-databank/A,_Dominique",48.33


In [53]:
astro4['lat_2'] = astro4['lat_2'].astype(float)

In [54]:
astro4.dtypes

nome        object
data        object
horario     object
lat         object
long        object
bio         object
url         object
lat_2      float64
dtype: object

Longitude:

In [55]:
# Primeiro, os dados referentes ao oeste, que devem ser negativos:
astro4.loc[(astro4['long'].str.contains("w")),'long_2'] = astro4['long'].str.replace('w','.')
astro4['long_2'] = "-" + astro4['long_2']

In [56]:
# Depois, os dados referentes ao leste:
astro4.loc[(astro4['long'].str.contains("e")),'long_2'] = astro4['long'].str.replace('e','.')

In [57]:
astro4.head()

,nome,data,horario,lat,long,bio,url,lat_2,long_2
0,"Babilée, Jean",1923/02/03,21:30,48n5102,2e1958,"French dancer, choreographer, and actor, Babil...",https://www.astro.com/astro-databank/Babil%C3%...,48.5102,2.1958
1,"Baber, Alice",1928/08/22,03:40,39n30,88w11,"American artist, her style was abstract expres...","https://www.astro.com/astro-databank/Baber,_Alice",39.3000,-88.11
2,"Babbitt, Manuel",1949/05/03,21:30,41n46,70w43,American homicide: a former Marine and Vietnam...,"https://www.astro.com/astro-databank/Babbitt,_...",41.4600,-70.43
3,"Babbitt, Elwood",1921/11/26,10:20,42n35,72w19,American medium based in New England. His lif...,"https://www.astro.com/astro-databank/Babbitt,_...",42.3500,-72.19
4,"A, Dominique",1968/10/06,17:35,48n33,3e18,"French songwriter, singer and guitarist.","https://www.astro.com/astro-databank/A,_Dominique",48.3300,3.18


In [58]:
astro4['long_2'] = astro4['long_2'].astype(float)

In [59]:
astro4.dtypes

nome        object
data        object
horario     object
lat         object
long        object
bio         object
url         object
lat_2      float64
long_2     float64
dtype: object

In [60]:
# Pode-se excluir as colunas originais:
astro4.drop(['lat', 'long'], axis=1, inplace=True)

Em seguida, determinou-se o fuso horário, em deslocamento UTC:

In [61]:
astro4['fuso'] = astro4.apply(offset, axis=1)

In [62]:
astro4.head()

,nome,data,horario,bio,url,lat_2,long_2,fuso
0,"Babilée, Jean",1923/02/03,21:30,"French dancer, choreographer, and actor, Babil...",https://www.astro.com/astro-databank/Babil%C3%...,48.5102,2.1958,1.0
1,"Baber, Alice",1928/08/22,03:40,"American artist, her style was abstract expres...","https://www.astro.com/astro-databank/Baber,_Alice",39.3000,-88.1100,-6.0
2,"Babbitt, Manuel",1949/05/03,21:30,American homicide: a former Marine and Vietnam...,"https://www.astro.com/astro-databank/Babbitt,_...",41.4600,-70.4300,-5.0
3,"Babbitt, Elwood",1921/11/26,10:20,American medium based in New England. His lif...,"https://www.astro.com/astro-databank/Babbitt,_...",42.3500,-72.1900,-5.0
4,"A, Dominique",1968/10/06,17:35,"French songwriter, singer and guitarist.","https://www.astro.com/astro-databank/A,_Dominique",48.3300,3.1800,1.0


In [63]:
astro4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43613 entries, 0 to 44499
Data columns (total 8 columns):
nome       43613 non-null object
data       43613 non-null object
horario    43613 non-null object
bio        43613 non-null object
url        43613 non-null object
lat_2      43613 non-null float64
long_2     43613 non-null float64
fuso       42978 non-null float64
dtypes: float64(3), object(5)
memory usage: 4.2+ MB


In [64]:
astro4 = astro4.dropna()

O dataset já está como o desejado; desta forma, atribuiu-se um nome definitivo para facilitar sua identificação:

In [65]:
df_astro = astro4

In [66]:
df_astro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42978 entries, 0 to 44499
Data columns (total 8 columns):
nome       42978 non-null object
data       42978 non-null object
horario    42978 non-null object
bio        42978 non-null object
url        42978 non-null object
lat_2      42978 non-null float64
long_2     42978 non-null float64
fuso       42978 non-null float64
dtypes: float64(3), object(5)
memory usage: 3.0+ MB


<div class="alert alert-block alert-info">
<b>Observação:</b> Query de consulta abaixo

In [67]:
df_astro[df_astro['nome'].str.contains('Neymar')]

,nome,data,horario,bio,url,lat_2,long_2,fuso
16678,Neymar,1992/02/05,02:15,Brazilian footballer who plays as a forward fo...,https://www.astro.com/astro-databank/Neymar,-23.31,-46.11,-3.0


***

### 2. Mapa astral:

Calculou-se o mapa astral para todos os elementos do dataset:

In [68]:
df_astro['mapa'] = df_astro.apply(mapa, axis=1)

In [69]:
df_astro.head()

,nome,data,horario,bio,url,lat_2,long_2,fuso,mapa
0,"Babilée, Jean",1923/02/03,21:30,"French dancer, choreographer, and actor, Babil...",https://www.astro.com/astro-databank/Babil%C3%...,48.5102,2.1958,1.0,"{'Sun': 'Aquarius', 'Moon': 'Virgo', 'Mercury'..."
1,"Baber, Alice",1928/08/22,03:40,"American artist, her style was abstract expres...","https://www.astro.com/astro-databank/Baber,_Alice",39.3000,-88.1100,-6.0,"{'Sun': 'Leo', 'Moon': 'Scorpio', 'Mercury': '..."
2,"Babbitt, Manuel",1949/05/03,21:30,American homicide: a former Marine and Vietnam...,"https://www.astro.com/astro-databank/Babbitt,_...",41.4600,-70.4300,-5.0,"{'Sun': 'Taurus', 'Moon': 'Cancer', 'Mercury':..."
3,"Babbitt, Elwood",1921/11/26,10:20,American medium based in New England. His lif...,"https://www.astro.com/astro-databank/Babbitt,_...",42.3500,-72.1900,-5.0,"{'Sun': 'Sagittarius', 'Moon': 'Libra', 'Mercu..."
4,"A, Dominique",1968/10/06,17:35,"French songwriter, singer and guitarist.","https://www.astro.com/astro-databank/A,_Dominique",48.3300,3.1800,1.0,"{'Sun': 'Libra', 'Moon': 'Aries', 'Mercury': '..."


In [70]:
# Para converter as chaves do dicionário em colunas, com seus respectivos valores:
df_astro = pd.concat([df_astro.drop(['mapa'], axis=1), df_astro['mapa'].apply(pd.Series)], axis=1)

In [71]:
df_astro.head()

,nome,data,horario,bio,url,lat_2,long_2,fuso,Sun,Moon,Mercury,Venus,Mars,Jupiter,Saturn,North Node,South Node,Syzygy,Pars Fortuna
0,"Babilée, Jean",1923/02/03,21:30,"French dancer, choreographer, and actor, Babil...",https://www.astro.com/astro-databank/Babil%C3%...,48.5102,2.1958,1.0,Aquarius,Virgo,Aquarius,Sagittarius,Aries,Scorpio,Libra,Virgo,Pisces,Leo,Aquarius
1,"Baber, Alice",1928/08/22,03:40,"American artist, her style was abstract expres...","https://www.astro.com/astro-databank/Baber,_Alice",39.3000,-88.1100,-6.0,Leo,Scorpio,Virgo,Virgo,Gemini,Taurus,Sagittarius,Gemini,Sagittarius,Leo,Taurus
2,"Babbitt, Manuel",1949/05/03,21:30,American homicide: a former Marine and Vietnam...,"https://www.astro.com/astro-databank/Babbitt,_...",41.4600,-70.4300,-5.0,Taurus,Cancer,Gemini,Taurus,Taurus,Aquarius,Leo,Aries,Libra,Taurus,Libra
3,"Babbitt, Elwood",1921/11/26,10:20,American medium based in New England. His lif...,"https://www.astro.com/astro-databank/Babbitt,_...",42.3500,-72.1900,-5.0,Sagittarius,Libra,Scorpio,Scorpio,Libra,Libra,Libra,Libra,Aries,Taurus,Sagittarius
4,"A, Dominique",1968/10/06,17:35,"French songwriter, singer and guitarist.","https://www.astro.com/astro-databank/A,_Dominique",48.3300,3.1800,1.0,Libra,Aries,Scorpio,Scorpio,Virgo,Virgo,Aries,Aries,Libra,Aries,Virgo


Com o mapa astral calculado para todas as linhas do dataset, eliminou-se as colunas "data", "horario", "bio", "url", "lat_2", "long_2" e "fuso".

In [72]:
df_astro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42978 entries, 0 to 44499
Data columns (total 19 columns):
nome            42978 non-null object
data            42978 non-null object
horario         42978 non-null object
bio             42978 non-null object
url             42978 non-null object
lat_2           42978 non-null float64
long_2          42978 non-null float64
fuso            42978 non-null float64
Sun             42978 non-null object
Moon            42978 non-null object
Mercury         42978 non-null object
Venus           42978 non-null object
Mars            42978 non-null object
Jupiter         42978 non-null object
Saturn          42978 non-null object
North Node      42978 non-null object
South Node      42978 non-null object
Syzygy          42978 non-null object
Pars Fortuna    42978 non-null object
dtypes: float64(3), object(16)
memory usage: 6.6+ MB


In [73]:
df_astro.drop(["data", "horario", "bio", "url", "lat_2", "long_2", "fuso"], axis=1, inplace=True)

In [74]:
df_astro.head()

,nome,Sun,Moon,Mercury,Venus,Mars,Jupiter,Saturn,North Node,South Node,Syzygy,Pars Fortuna
0,"Babilée, Jean",Aquarius,Virgo,Aquarius,Sagittarius,Aries,Scorpio,Libra,Virgo,Pisces,Leo,Aquarius
1,"Baber, Alice",Leo,Scorpio,Virgo,Virgo,Gemini,Taurus,Sagittarius,Gemini,Sagittarius,Leo,Taurus
2,"Babbitt, Manuel",Taurus,Cancer,Gemini,Taurus,Taurus,Aquarius,Leo,Aries,Libra,Taurus,Libra
3,"Babbitt, Elwood",Sagittarius,Libra,Scorpio,Scorpio,Libra,Libra,Libra,Libra,Aries,Taurus,Sagittarius
4,"A, Dominique",Libra,Aries,Scorpio,Scorpio,Virgo,Virgo,Aries,Aries,Libra,Aries,Virgo


In [75]:
df_astro.reset_index(inplace=True)
df_astro.drop("index", axis=1, inplace=True)

In [76]:
df_astro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42978 entries, 0 to 42977
Data columns (total 12 columns):
nome            42978 non-null object
Sun             42978 non-null object
Moon            42978 non-null object
Mercury         42978 non-null object
Venus           42978 non-null object
Mars            42978 non-null object
Jupiter         42978 non-null object
Saturn          42978 non-null object
North Node      42978 non-null object
South Node      42978 non-null object
Syzygy          42978 non-null object
Pars Fortuna    42978 non-null object
dtypes: object(12)
memory usage: 3.9+ MB


<div class="alert alert-block alert-info">
<b>Observação:</b> Dois indivíduos que possuem mapa astral exatamente igual

In [77]:
df_astro_semnome = df_astro.drop('nome', axis=1)

In [78]:
df_astro_semnome[(df_astro_semnome["Sun"]=="Cancer")&(df_astro_semnome["Moon"]=="Cancer")&(df_astro_semnome["Mercury"]=="Leo")][:2]

,Sun,Moon,Mercury,Venus,Mars,Jupiter,Saturn,North Node,South Node,Syzygy,Pars Fortuna
24,Cancer,Cancer,Leo,Leo,Gemini,Gemini,Leo,Capricorn,Cancer,Capricorn,Taurus
52,Cancer,Cancer,Leo,Leo,Gemini,Gemini,Leo,Capricorn,Cancer,Capricorn,Taurus


***

### 3. Cálculo de distâncias:

#### 3.1 Por similaridade:

Calculou-se as distâncias entre os indivíduos do dataset. <br>
Para isso, primeiramente determinou-se dummies para as colunas categóricas.

In [79]:
df_astro_dummies = pd.get_dummies(df_astro, columns=['Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter',
                                                    'Saturn','North Node', 'South Node', 'Syzygy', 'Pars Fortuna'])

In [80]:
df_astro_dummies.head()

,nome,Sun_Aquarius,Sun_Aries,Sun_Cancer,Sun_Capricorn,Sun_Gemini,Sun_Leo,Sun_Libra,Sun_Pisces,Sun_Sagittarius,...,Pars Fortuna_Cancer,Pars Fortuna_Capricorn,Pars Fortuna_Gemini,Pars Fortuna_Leo,Pars Fortuna_Libra,Pars Fortuna_Pisces,Pars Fortuna_Sagittarius,Pars Fortuna_Scorpio,Pars Fortuna_Taurus,Pars Fortuna_Virgo
0,"Babilée, Jean",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Baber, Alice",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,"Babbitt, Manuel",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,"Babbitt, Elwood",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,"A, Dominique",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


Em seguida, utilizou-se o algoritmo BallTree para calcular as distâncias entre os k vizinhos mais próximos em relação a um determinado ponto. 

Como exemplo, verificou-se quais são os 10 indivíduos que possuem o mapa astral que mais se assemelha ao do Neymar.

In [81]:
def top10_semelhantes(nome_do_usuario):
    tree = BallTree(df_astro_dummies.drop('nome', axis=1), leaf_size=2)
    dist, ind = tree.query(df_astro_dummies.loc[df_astro_dummies["nome"]==nome_do_usuario].drop('nome', axis=1), k=11) 
    
    ind_list=[]
    for ind1 in ind:
        for i in ind1:
            ind_list.append(i)
    ind_list.remove(df_astro_dummies.loc[df_astro_dummies["nome"]==nome_do_usuario].index)
    
    dist_list=[]
    for dis in dist:
        for d in dis:
            dist_list.append(d)
    dist_list.remove(0)
    
    df_igual=pd.DataFrame()
    df_semelhante=pd.DataFrame()
    for n,d in enumerate(dist_list):
        if d == 0:
            df_igual = df_igual.append(df_astro.loc[[ind_list[n]]])
        elif d!=0: 
            df_semelhante = df_semelhante.append(df_astro.loc[[ind_list[n]]])
    
    porcentagem = []
    for index, row in df_semelhante.iterrows():
        for index2, row2 in df_astro.loc[df_astro["nome"]==nome_do_usuario].iterrows():
            comparacao = row==row2
            porcentagem.append(round((((comparacao==True).value_counts(sort=False).values[1])/11)*100,2))
            
    print('-------------------------------------------------------------')
    print('Olá, %s!' %nome_do_usuario)
    print('\n')
    print('Você é idêntico às seguintes pessoas:')
    print('>>','\n>> '.join([i for i in df_igual['nome']]))
    print('\n')
    print('Você é similar às seguintes pessoas:')
    print('>>','\n>> '.join([i+" - "+str(p)+"%" for i,p in zip(df_semelhante['nome'],porcentagem)]))
    print('-------------------------------------------------------------')

In [82]:
top10_semelhantes("Neymar")

-------------------------------------------------------------
Olá, Neymar!


Você é idêntico às seguintes pessoas:
>> Benattia, Nabilla


Você é similar às seguintes pessoas:
>> Wagner, Bryan Matthew - 90.91%
>> Jain (singer) - 90.91%
>> Efron, Dylan - 90.91%
>> Pino, Andrea - 81.82%
>> Stal, Gary - 81.82%
>> Ferrand-Prévot, Pauline - 81.82%
>> Lautner, Taylor - 81.82%
>> Sutton, Chloe - 81.82%
>> Fukuhara, Karen - 81.82%
-------------------------------------------------------------


In [83]:
tree = BallTree(df_astro_dummies.drop('nome', axis=1), leaf_size=2)

In [84]:
dist, ind = tree.query(df_astro_dummies.loc[df_astro_dummies["nome"]=="Neymar"].drop('nome', axis=1), k=11) 

In [85]:
ind_list=[]
for ind1 in ind:
    for i in ind1:
        ind_list.append(i)

ind_list

[36834, 16110, 3685, 2157, 1882, 15172, 8851, 28181, 21485, 39462, 27032]

In [86]:
ind_list.remove(df_astro_dummies.loc[df_astro_dummies["nome"]=="Neymar"].index)

In [87]:
ind_list

[36834, 3685, 2157, 1882, 15172, 8851, 28181, 21485, 39462, 27032]

In [88]:
df_astro_dummies.iloc[ind_list]

,nome,Sun_Aquarius,Sun_Aries,Sun_Cancer,Sun_Capricorn,Sun_Gemini,Sun_Leo,Sun_Libra,Sun_Pisces,Sun_Sagittarius,...,Pars Fortuna_Cancer,Pars Fortuna_Capricorn,Pars Fortuna_Gemini,Pars Fortuna_Leo,Pars Fortuna_Libra,Pars Fortuna_Pisces,Pars Fortuna_Sagittarius,Pars Fortuna_Scorpio,Pars Fortuna_Taurus,Pars Fortuna_Virgo
36834,"Benattia, Nabilla",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3685,"Wagner, Bryan Matthew",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2157,Jain (singer),1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1882,"Efron, Dylan",1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
15172,"Pino, Andrea",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8851,"Stal, Gary",1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
28181,"Ferrand-Prévot, Pauline",1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
21485,"Lautner, Taylor",1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
39462,"Sutton, Chloe",1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
27032,"Fukuhara, Karen",1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [89]:
dist_list=[]
for dis in dist:
    for d in dis:
        dist_list.append(d)

dist_list

[0.0,
 0.0,
 1.4142135623730951,
 1.4142135623730951,
 1.4142135623730951,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0]

In [90]:
dist_list.remove(0)

In [91]:
dist_list

[0.0,
 1.4142135623730951,
 1.4142135623730951,
 1.4142135623730951,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0]

In [92]:
df_neymar_igual=pd.DataFrame()
df_neymar_semelhante=pd.DataFrame()
for n,d in enumerate(dist_list):
    if d == 0:
        df_neymar_igual = df_neymar_igual.append(df_astro.loc[[ind_list[n]]])
    elif d!=0: 
        df_neymar_semelhante = df_neymar_semelhante.append(df_astro.loc[[ind_list[n]]])

In [93]:
df_neymar_igual

,nome,Sun,Moon,Mercury,Venus,Mars,Jupiter,Saturn,North Node,South Node,Syzygy,Pars Fortuna
36834,"Benattia, Nabilla",Aquarius,Pisces,Aquarius,Capricorn,Capricorn,Virgo,Aquarius,Capricorn,Cancer,Aquarius,Sagittarius


In [94]:
df_neymar_semelhante

,nome,Sun,Moon,Mercury,Venus,Mars,Jupiter,Saturn,North Node,South Node,Syzygy,Pars Fortuna
3685,"Wagner, Bryan Matthew",Aquarius,Gemini,Aquarius,Capricorn,Capricorn,Virgo,Aquarius,Capricorn,Cancer,Aquarius,Sagittarius
2157,Jain (singer),Aquarius,Pisces,Aquarius,Capricorn,Capricorn,Virgo,Aquarius,Capricorn,Cancer,Aquarius,Cancer
1882,"Efron, Dylan",Aquarius,Pisces,Aquarius,Capricorn,Capricorn,Virgo,Aquarius,Capricorn,Cancer,Aquarius,Leo
15172,"Pino, Andrea",Aquarius,Cancer,Aquarius,Capricorn,Capricorn,Virgo,Aquarius,Capricorn,Cancer,Aquarius,Virgo
8851,"Stal, Gary",Aquarius,Aries,Aquarius,Capricorn,Capricorn,Virgo,Aquarius,Capricorn,Cancer,Aquarius,Leo
28181,"Ferrand-Prévot, Pauline",Aquarius,Taurus,Aquarius,Capricorn,Capricorn,Virgo,Aquarius,Capricorn,Cancer,Aquarius,Leo
21485,"Lautner, Taylor",Aquarius,Taurus,Aquarius,Capricorn,Capricorn,Virgo,Aquarius,Capricorn,Cancer,Aquarius,Leo
39462,"Sutton, Chloe",Aquarius,Aquarius,Aquarius,Capricorn,Capricorn,Virgo,Aquarius,Capricorn,Cancer,Aquarius,Leo
27032,"Fukuhara, Karen",Aquarius,Taurus,Aquarius,Capricorn,Capricorn,Virgo,Aquarius,Capricorn,Cancer,Aquarius,Cancer


In [95]:
df_astro.loc[df_astro["nome"]=="Neymar"]

,nome,Sun,Moon,Mercury,Venus,Mars,Jupiter,Saturn,North Node,South Node,Syzygy,Pars Fortuna
16110,Neymar,Aquarius,Pisces,Aquarius,Capricorn,Capricorn,Virgo,Aquarius,Capricorn,Cancer,Aquarius,Sagittarius


In [96]:
#calculo de porcentagem de semelhança

for index, row in df_astro.loc[df_astro["nome"].str.contains("Bolsonaro")].iterrows():
        for index2, row2 in df_astro.loc[df_astro["nome"]=="Neymar"].iterrows():
            comparacao = row==row2
            porcentagem_semelhanca=(round((((comparacao==True).value_counts(sort=False).values[1])/11)*100,2))

In [97]:
#calculo de porcentagem de semelhança

for index, row in df_astro.loc[df_astro["nome"].str.contains("Bolsonaro")].iterrows():
        for index2, row2 in df_astro.loc[df_astro["nome"]=="Neymar"].iterrows():
            comparacao = row==row2
            porcentagem_semelhanca=((comparacao==True).values.sum())

In [98]:
#calculo de porcentagem de compatibilidade
porcentagem_compat = []
for index, row in (df_astro.loc[df_astro["nome"].str.contains("Bolsonaro")].drop('nome', axis=1)).iterrows():
        for index2, row2 in (df_astro.loc[df_astro["nome"]=="Neymar"].drop('nome', axis=1)).iterrows():
            comparacao = row==row2
            porcentagem_compat.append((comparacao==True).value_counts(sort=False).values[0]/11*100)

In [99]:
porcentagem_semelhanca

2

In [100]:
porcentagem_compat

[81.81818181818183]

***

#### 3.2 Por compatibilidade:

A compatibilidade foi determinada por complementaridade. Dessa forma, os signos da base de dados original foram substituídos pelos seus opostos (180 °) no ciclo astrológico.

In [101]:
df_astro.head()

,nome,Sun,Moon,Mercury,Venus,Mars,Jupiter,Saturn,North Node,South Node,Syzygy,Pars Fortuna
0,"Babilée, Jean",Aquarius,Virgo,Aquarius,Sagittarius,Aries,Scorpio,Libra,Virgo,Pisces,Leo,Aquarius
1,"Baber, Alice",Leo,Scorpio,Virgo,Virgo,Gemini,Taurus,Sagittarius,Gemini,Sagittarius,Leo,Taurus
2,"Babbitt, Manuel",Taurus,Cancer,Gemini,Taurus,Taurus,Aquarius,Leo,Aries,Libra,Taurus,Libra
3,"Babbitt, Elwood",Sagittarius,Libra,Scorpio,Scorpio,Libra,Libra,Libra,Libra,Aries,Taurus,Sagittarius
4,"A, Dominique",Libra,Aries,Scorpio,Scorpio,Virgo,Virgo,Aries,Aries,Libra,Aries,Virgo


In [102]:
df_astro_compat = df_astro.copy()

In [103]:
#1
df_astro_compat["Sun"] = df_astro_compat["Sun"].map({
    'Aries':'Libra','Taurus':'Scorpio','Gemini':'Sagittarius','Cancer':'Capricorn','Leo':'Aquarius','Virgo':'Pisces',
    'Libra':'Aries','Scorpio':'Taurus','Sagittarius':'Gemini','Capricorn':'Cancer','Aquarius':'Leo','Pisces':'Virgo'
    })

#2
df_astro_compat["Moon"] = df_astro_compat["Moon"].map({
    'Aries':'Libra','Taurus':'Scorpio','Gemini':'Sagittarius','Cancer':'Capricorn','Leo':'Aquarius','Virgo':'Pisces',
    'Libra':'Aries','Scorpio':'Taurus','Sagittarius':'Gemini','Capricorn':'Cancer','Aquarius':'Leo','Pisces':'Virgo'
    })

#3
df_astro_compat["Mercury"] = df_astro_compat["Mercury"].map({
    'Aries':'Libra','Taurus':'Scorpio','Gemini':'Sagittarius','Cancer':'Capricorn','Leo':'Aquarius','Virgo':'Pisces',
    'Libra':'Aries','Scorpio':'Taurus','Sagittarius':'Gemini','Capricorn':'Cancer','Aquarius':'Leo','Pisces':'Virgo'
    })

#4
df_astro_compat["Venus"] = df_astro_compat["Venus"].map({
    'Aries':'Libra','Taurus':'Scorpio','Gemini':'Sagittarius','Cancer':'Capricorn','Leo':'Aquarius','Virgo':'Pisces',
    'Libra':'Aries','Scorpio':'Taurus','Sagittarius':'Gemini','Capricorn':'Cancer','Aquarius':'Leo','Pisces':'Virgo'
    })

#5
df_astro_compat["Mars"] = df_astro_compat["Mars"].map({
    'Aries':'Libra','Taurus':'Scorpio','Gemini':'Sagittarius','Cancer':'Capricorn','Leo':'Aquarius','Virgo':'Pisces',
    'Libra':'Aries','Scorpio':'Taurus','Sagittarius':'Gemini','Capricorn':'Cancer','Aquarius':'Leo','Pisces':'Virgo'
    })

#6
df_astro_compat["Jupiter"] = df_astro_compat["Jupiter"].map({
    'Aries':'Libra','Taurus':'Scorpio','Gemini':'Sagittarius','Cancer':'Capricorn','Leo':'Aquarius','Virgo':'Pisces',
    'Libra':'Aries','Scorpio':'Taurus','Sagittarius':'Gemini','Capricorn':'Cancer','Aquarius':'Leo','Pisces':'Virgo'
    })

#7
df_astro_compat["Saturn"] = df_astro_compat["Saturn"].map({
    'Aries':'Libra','Taurus':'Scorpio','Gemini':'Sagittarius','Cancer':'Capricorn','Leo':'Aquarius','Virgo':'Pisces',
    'Libra':'Aries','Scorpio':'Taurus','Sagittarius':'Gemini','Capricorn':'Cancer','Aquarius':'Leo','Pisces':'Virgo'
    })

#8
df_astro_compat["North Node"] = df_astro_compat["North Node"].map({
    'Aries':'Libra','Taurus':'Scorpio','Gemini':'Sagittarius','Cancer':'Capricorn','Leo':'Aquarius','Virgo':'Pisces',
    'Libra':'Aries','Scorpio':'Taurus','Sagittarius':'Gemini','Capricorn':'Cancer','Aquarius':'Leo','Pisces':'Virgo'
    })

#9
df_astro_compat["South Node"] = df_astro_compat["South Node"].map({
    'Aries':'Libra','Taurus':'Scorpio','Gemini':'Sagittarius','Cancer':'Capricorn','Leo':'Aquarius','Virgo':'Pisces',
    'Libra':'Aries','Scorpio':'Taurus','Sagittarius':'Gemini','Capricorn':'Cancer','Aquarius':'Leo','Pisces':'Virgo'
    })

#10
df_astro_compat["Syzygy"] = df_astro_compat["Syzygy"].map({
    'Aries':'Libra','Taurus':'Scorpio','Gemini':'Sagittarius','Cancer':'Capricorn','Leo':'Aquarius','Virgo':'Pisces',
    'Libra':'Aries','Scorpio':'Taurus','Sagittarius':'Gemini','Capricorn':'Cancer','Aquarius':'Leo','Pisces':'Virgo'
    })

#11
df_astro_compat["Pars Fortuna"] = df_astro_compat["Pars Fortuna"].map({
    'Aries':'Libra','Taurus':'Scorpio','Gemini':'Sagittarius','Cancer':'Capricorn','Leo':'Aquarius','Virgo':'Pisces',
    'Libra':'Aries','Scorpio':'Taurus','Sagittarius':'Gemini','Capricorn':'Cancer','Aquarius':'Leo','Pisces':'Virgo'
    })

In [104]:
df_astro_compat.head()

,nome,Sun,Moon,Mercury,Venus,Mars,Jupiter,Saturn,North Node,South Node,Syzygy,Pars Fortuna
0,"Babilée, Jean",Leo,Pisces,Leo,Gemini,Libra,Taurus,Aries,Pisces,Virgo,Aquarius,Leo
1,"Baber, Alice",Aquarius,Taurus,Pisces,Pisces,Sagittarius,Scorpio,Gemini,Sagittarius,Gemini,Aquarius,Scorpio
2,"Babbitt, Manuel",Scorpio,Capricorn,Sagittarius,Scorpio,Scorpio,Leo,Aquarius,Libra,Aries,Scorpio,Aries
3,"Babbitt, Elwood",Gemini,Aries,Taurus,Taurus,Aries,Aries,Aries,Aries,Libra,Scorpio,Gemini
4,"A, Dominique",Aries,Libra,Taurus,Taurus,Pisces,Pisces,Libra,Libra,Aries,Libra,Pisces


Preparado o dataset, foram geradas dummies.

In [105]:
df_astro_compat_dummies = pd.get_dummies(df_astro_compat, columns=['Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter',
                                                    'Saturn','North Node', 'South Node', 'Syzygy', 'Pars Fortuna'])

Esse dataset de compatibilidade não permite que seja feito o teste para uma personalidade que já está nele (como o Neymar, por exemplo), pois toda a base foi modificada. Sendo assim, a compatibilidade deve ser estabelecida entre os dados do usuário e a base original.

In [106]:
tree = BallTree(df_astro_dummies.drop('nome', axis=1), leaf_size=2)

In [107]:
dist, ind = tree.query(df_astro_dummies.loc[df_astro_dummies["nome"]=="Neymar"].drop('nome', axis=1), k=11) 

***

### 4. Interação com os dados do usuário:

In [108]:
def usuario():
    """
    Essa função coleta os dados que o usuário fornece e retorna as 10 personalidades mais semelhantes e as 10 mais compatíveis
    """
    #1. Zerar a base, através de uma cópia da base original
    df_astro_copia = df_astro.copy()
    df_astro_dummies_copia = df_astro_dummies.copy()
    df_astro_compat_dummies_copia = df_astro_compat_dummies.copy()
    
    #2. Coletar os dados
    us_nome = input('Qual o seu nome?')
    
    checker_data = False
    checker_hora = False
    checker_fuso = False
    while checker_data == False:
        us_data = input('Em que dia você nasceu? (AAAA/MM/DD)')
        if re.search('^\d\d\d\d\/(0?[1-9]|1[0-2])\/(0?[1-9]|[12][0-9]|3[01])', us_data):
            checker_data = True
        else:
            print('Valor inválido. Insira novamente.')
    
    while checker_hora == False:
        us_hora = input('Em qual horário você nasceu? (HH:MM)')
        if re.search('(0[0-9]|1[0-9]|2[0-3]):[0-5][0-9]', us_hora):
            checker_hora = True
        else:
            print('Valor inválido. Insira novamente.')
            
    while checker_fuso == False:
        us_fuso = input('Em qual fuso horário (UTC)? (-03:00 para paulistas, fluminenses e soteropolitanos)')
        if re.search('-?(0[0-9]|1[0-9]|2[0-3]):[0-5][0-9]', us_fuso):
            checker_fuso = True
        else:
            print('Valor inválido. Insira novamente.')
            
    us_cidade = input('Em qual cidade você nasceu? (Cidade, UF)')
    user = pessoa(us_nome, us_data, us_hora, us_fuso, us_cidade)
    user.mapa()
    dicio = user.chart
    
    #3. Armazenar os dados em um dataframe
    df_user = pd.DataFrame([dicio])
    df_user = df_user[['nome', 'Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn',
                       'North Node', 'South Node', 'Syzygy', 'Pars Fortuna']]
    df_user_dummies = pd.get_dummies(df_user, columns=['Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter',
                        'Saturn','North Node', 'South Node', 'Syzygy', 'Pars Fortuna'])
    
    #4. Juntar o dataframe do usuário com a base de dados
    df_astro_user = df_user.append(df_astro_copia, sort=False)
    df_astro_user_dummies = df_user_dummies.append(df_astro_dummies_copia, sort=False)
    df_astro_user_dummies.fillna(0, inplace=True)
    df_astro_user_compat_dummies = df_user_dummies.append(df_astro_compat_dummies_copia, sort=False)
    df_astro_user_compat_dummies.fillna(0, inplace=True)
    
    df_astro_user.reset_index(inplace=True)
    df_astro_user.drop("index", axis=1, inplace=True)
    df_astro_user_dummies.reset_index(inplace=True)
    df_astro_user_dummies.drop("index", axis=1, inplace=True)
    df_astro_user_compat_dummies.reset_index(inplace=True)
    df_astro_user_compat_dummies.drop("index", axis=1, inplace=True)
    
    #5. Cálculos: similaridade
    tree = BallTree(df_astro_user_dummies.drop('nome', axis=1), leaf_size=2)
    dist, ind = tree.query(df_astro_user_dummies.loc[df_astro_user_dummies["nome"]==us_nome].drop('nome', axis=1), k=11) 
    
    ind_list=[]
    for ind1 in ind:
        for i in ind1:
            ind_list.append(i)
    ind_list.remove(df_astro_user_dummies.loc[df_astro_user_dummies["nome"]==us_nome].index)
    
    dist_list=[]
    for dis in dist:
        for d in dis:
            dist_list.append(d)
    dist_list.remove(0)
    
    df_igual=pd.DataFrame()
    df_semelhante=pd.DataFrame()
    for n,d in enumerate(dist_list):
        if d == 0:
            df_igual = df_igual.append(df_astro_user.loc[[ind_list[n]]])
        elif d!=0: 
            df_semelhante = df_semelhante.append(df_astro_user.loc[[ind_list[n]]])
    
    porcentagem = []
    for index, row in df_semelhante.iterrows():
        for index2, row2 in df_astro_user.loc[df_astro_user["nome"]==us_nome].iterrows():
            comparacao = row==row2
            porcentagem.append(round((((comparacao==True).value_counts(sort=False).values[1])/11)*100,2))
            
    #6. Cálculos: compatibilidade
    tree_compat = BallTree(df_astro_user_compat_dummies.drop('nome', axis=1), leaf_size=2)
    dist2, ind2 = tree_compat.query(df_astro_user_compat_dummies.loc[df_astro_user_compat_dummies["nome"]==us_nome].drop('nome', axis=1), k=11) 
    
    ind_list2=[]
    for ind1_2 in ind2:
        for i in ind1_2:
            ind_list2.append(i)
    ind_list2.remove(df_astro_user_compat_dummies.loc[df_astro_user_compat_dummies["nome"]==us_nome].index)
    
    dist_list2=[]
    for dis in dist2:
        for d in dis:
            dist_list2.append(d)
    dist_list2.remove(0)
    
    df_comp = df_astro_user.iloc[ind_list2]
    
    porcentagem2 = []
    for index, row in (df_comp.drop('nome', axis=1)).iterrows():
        for index2, row2 in (df_astro_user.loc[df_astro_user["nome"]==us_nome].drop('nome', axis=1)).iterrows():
            comparacao2 = row==row2
            porcentagem2.append(round((((comparacao2==True).value_counts(sort=False).values[0])/11)*100,2))
    porcentagem2.sort(reverse=True)
    
    #7. Outputs
            
    print('-------------------------------------------------------------')
    print('Olá, %s!' %us_nome)
    print('\n')
    if df_igual.empty == True:
        print('Você não é idêntico a ninguém!')
    else:
        print('Você é idêntico às seguintes pessoas:')
        print('>>','\n>> '.join([i for i in df_igual['nome']]))
    print('\n')
    print('Você é similar às seguintes pessoas:')
    print('>>','\n>> '.join([i+" - "+str(p)+"%" for i,p in zip(df_semelhante['nome'],porcentagem)]))
    print('-------------------------------------------------------------')
    print('\n')
    print('Você é compatível às seguintes pessoas:')
    print('>>','\n>> '.join([i+" - "+str(p)+"%" for i,p in zip(df_comp['nome'],porcentagem2)]))
    print('-------------------------------------------------------------')

In [109]:
usuario()

Qual o seu nome?fer
Em que dia você nasceu? (AAAA/MM/DD)1992/08/15
Em qual horário você nasceu? (HH:MM)13:30
Em qual fuso horário (UTC)? (-03:00 para paulistas, fluminenses e soteropolitanos)-03:00
Em qual cidade você nasceu? (Cidade, UF)São Paulo, SP
-------------------------------------------------------------
Olá, fer!


Você não é idêntico a ninguém!


Você é similar às seguintes pessoas:
>> Mikac, Madeline - 90.91%
>> Lovato, Demi - 81.82%
>> Cobain, Frances Bean - 81.82%
>> Mossely, Estelle - 81.82%
>> CHK - 81.82%
>> Powers, Keith - 81.82%
>> Stone, Spencer - 81.82%
>> Petras, Kim - 72.73%
>> Anysio, Rodrigo de Paula - 72.73%
>> Weaver, Ryn - 72.73%
-------------------------------------------------------------


Você é compatível às seguintes pessoas:
>> Knies, Beatrice Urban - 100.0%
>> Margerit, Robert - 100.0%
>> Hichtum, Nienke van - 100.0%
>> Grieshaber, HAP - 100.0%
>> Zavala, Silvio - 100.0%
>> Ouvrard, Gaston - 100.0%
>> Charner, Léonard - 90.91%
>> Ladd, David - 90.91%


In [133]:
def sou_semelhante(personalidade):
    
    #1. Zerar a base, através de uma cópia da base original
    df_astro_copia = df_astro.copy()
    
    if df_astro_copia.loc[df_astro_copia["nome"].str.contains(personalidade)].empty == True:
        print('Não há dados sobre essa personalidade!')
    else:
    
        #2. Coletar os dados
        us_nome = input('Qual o seu nome?')

        checker_data = False
        checker_hora = False
        checker_fuso = False
        while checker_data == False:
            us_data = input('Em que dia você nasceu? (AAAA/MM/DD)')
            if re.search('^\d\d\d\d\/(0?[1-9]|1[0-2])\/(0?[1-9]|[12][0-9]|3[01])', us_data):
                checker_data = True
            else:
                print('Valor inválido. Insira novamente.')

        while checker_hora == False:
            us_hora = input('Em qual horário você nasceu? (HH:MM)')
            if re.search('(0[0-9]|1[0-9]|2[0-3]):[0-5][0-9]', us_hora):
                checker_hora = True
            else:
                print('Valor inválido. Insira novamente.')

        while checker_fuso == False:
            us_fuso = input('Em qual fuso horário (UTC)? (-03:00 para paulistas, fluminenses e soteropolitanos)')
            if re.search('-?(0[0-9]|1[0-9]|2[0-3]):[0-5][0-9]', us_fuso):
                checker_fuso = True
            else:
                print('Valor inválido. Insira novamente.')

        us_cidade = input('Em qual cidade você nasceu? (Cidade, UF)')
        user = pessoa(us_nome, us_data, us_hora, us_fuso, us_cidade)
        user.mapa()
        dicio = user.chart

        #3. Armazenar os dados em um dataframe
        df_user = pd.DataFrame([dicio])
        df_user = df_user[['nome', 'Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn',
                           'North Node', 'South Node', 'Syzygy', 'Pars Fortuna']]

        #4. Juntar o dataframe do usuário com a base de dados
        df_astro_user = df_user.append(df_astro_copia, sort=False)
        df_astro_user.reset_index(inplace=True)
        df_astro_user.drop("index", axis=1, inplace=True)

        for index, row in df_astro_user.loc[df_astro_user["nome"].str.contains(personalidade)].iterrows():
            for index2, row2 in df_astro_user.loc[df_astro_user["nome"]==us_nome].iterrows():
                comparacao = row==row2
                porcentagem_semelhanca=round(((comparacao==True).values.sum())/11*100,2)

        print('-------------------------------------------------------------')
        print('Olá, %s!' %us_nome)
        print('\n')
        print('Você é %s %% similar a %s.' %(porcentagem_semelhanca, personalidade))
        print('-------------------------------------------------------------')

In [135]:
sou_semelhante("Ninomiya,Kazunari")

Não há dados sobre essa personalidade!


In [130]:
def sou_compativel(personalidade):
    
    #1. Zerar a base, através de uma cópia da base original
    df_astro_copia = df_astro.copy()
    
    if df_astro_copia.loc[df_astro_copia["nome"].str.contains(personalidade)].empty == True:
        print('Não há dados sobre essa personalidade!')
        
    else:
        #2. Coletar os dados
        us_nome = input('Qual o seu nome?')

        checker_data = False
        checker_hora = False
        checker_fuso = False
        while checker_data == False:
            us_data = input('Em que dia você nasceu? (AAAA/MM/DD)')
            if re.search('^\d\d\d\d\/(0?[1-9]|1[0-2])\/(0?[1-9]|[12][0-9]|3[01])', us_data):
                checker_data = True
            else:
                print('Valor inválido. Insira novamente.')

        while checker_hora == False:
            us_hora = input('Em qual horário você nasceu? (HH:MM)')
            if re.search('(0[0-9]|1[0-9]|2[0-3]):[0-5][0-9]', us_hora):
                checker_hora = True
            else:
                print('Valor inválido. Insira novamente.')

        while checker_fuso == False:
            us_fuso = input('Em qual fuso horário (UTC)? (-03:00 para paulistas, fluminenses e soteropolitanos)')
            if re.search('-?(0[0-9]|1[0-9]|2[0-3]):[0-5][0-9]', us_fuso):
                checker_fuso = True
            else:
                print('Valor inválido. Insira novamente.')

        us_cidade = input('Em qual cidade você nasceu? (Cidade, UF)')
        user = pessoa(us_nome, us_data, us_hora, us_fuso, us_cidade)
        user.mapa()
        dicio = user.chart

        #3. Armazenar os dados em um dataframe
        df_user = pd.DataFrame([dicio])
        df_user = df_user[['nome', 'Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn',
                           'North Node', 'South Node', 'Syzygy', 'Pars Fortuna']]

        #4. Juntar o dataframe do usuário com a base de dados
        df_astro_user = df_user.append(df_astro_copia, sort=False)
        df_astro_user.reset_index(inplace=True)
        df_astro_user.drop("index", axis=1, inplace=True)

        for index, row in (df_astro_user.loc[df_astro_user["nome"].str.contains(personalidade)].drop('nome', axis=1)).iterrows():
            for index2, row2 in (df_astro_user.loc[df_astro_user["nome"]==us_nome].drop('nome', axis=1)).iterrows():
                comparacao = row==row2
                porcentagem_compat=round((comparacao==False).values.sum()/11*100,2)

        print('-------------------------------------------------------------')
        print('Olá, %s!' %us_nome)
        print('\n')
        print('Você é %s %% compatível com %s.' %(porcentagem_compat, personalidade))
        print('-------------------------------------------------------------')

In [132]:
sou_compativel("Neymar")

Qual o seu nome?fer
Em que dia você nasceu? (AAAA/MM/DD)1992/08/15
Em qual horário você nasceu? (HH:MM)13:30
Em qual fuso horário (UTC)? (-03:00 para paulistas, fluminenses e soteropolitanos)-03:00
Em qual cidade você nasceu? (Cidade, UF)São Paulo
-------------------------------------------------------------
Olá, fer!


Você é 63.64 % compatível com Neymar.
-------------------------------------------------------------
